In [ ]:
    from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import RepeatedStratifiedKFold
from tqdm.notebook import tqdm


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [ ]:
  !pip install optuna

     |████████████████████████████████| 308 kB 4.5 MB/s 
     |████████████████████████████████| 81 kB 8.2 MB/s 
     |████████████████████████████████| 210 kB 43.1 MB/s 
     |████████████████████████████████| 78 kB 3.5 MB/s 
     |████████████████████████████████| 146 kB 45.9 MB/s 
     |████████████████████████████████| 49 kB 4.0 MB/s 
     |████████████████████████████████| 113 kB 46.5 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=64370a483d9a17c688a3484670ef956058831f819565781de37e26af18bb2f7e
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
import optuna

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/project9/train_transformed.csv')
train_labels = pd.read_csv('/content/drive/MyDrive/project9/music30s_trainlabel.csv')
test =pd.read_csv('/content/drive/MyDrive/project9/test_transformed.csv')


In [ ]:
train_labels =train_labels.iloc[:,1:]
train_labels.head(3)


,label
0,reggae
1,reggae
2,country


In [ ]:
display(train.head(3))
display(test.head(3))

,length,chroma_stft_mean,rms_mean,spectral_centroid_mean,spectral_bandwidth_mean,rolloff_mean,zero_crossing_rate_mean,harmony_mean,perceptr_mean,tempo,...,mfcc11_var,mfcc12_var,mfcc13_var,mfcc14_var,mfcc15_var,mfcc16_var,mfcc17_var,mfcc18_var,mfcc19_var,mfcc20_var
0,661504,0.432846,0.142929,3089.637185,2973.703815,6574.771380,0.136156,-0.000001,-0.000059,112.347147,...,1.838597,0.833696,1.034494,0.776698,0.990850,1.481935,0.680471,0.666631,0.743034,-0.091657
1,661504,0.398761,0.092221,2170.544505,2054.465986,4306.240937,0.119993,-0.002001,-0.002011,143.554688,...,0.824710,-0.080086,0.785104,0.768889,1.027054,0.374397,0.127377,0.540703,-0.063098,0.060347
2,661794,0.375874,0.099977,2321.871533,2545.325000,4580.418732,0.104365,0.000002,0.000088,99.384014,...,0.631436,0.303991,-0.387986,0.346198,0.130285,-0.055250,-0.151197,-0.366009,-1.045659,-1.167858


,length,chroma_stft_mean,rms_mean,spectral_centroid_mean,spectral_bandwidth_mean,rolloff_mean,zero_crossing_rate_mean,harmony_mean,perceptr_mean,tempo,...,mfcc11_var,mfcc12_var,mfcc13_var,mfcc14_var,mfcc15_var,mfcc16_var,mfcc17_var,mfcc18_var,mfcc19_var,mfcc20_var
0,661504,0.326701,0.056816,1366.306035,1736.646891,2562.717631,0.071181,-2.218249e-03,-0.001156,129.199219,...,1.438900,1.531189,1.933873,1.593768,1.864126,0.803889,1.817925,1.818024,1.261632,1.923743
1,661794,0.437906,0.005276,971.777391,1614.749295,1906.571121,0.031534,-8.984312e-06,-0.000014,143.554688,...,-1.153414,-1.881063,-3.674341,-3.609437,-3.750683,-3.791344,-3.700374,-3.402824,-2.418003,-1.934408
2,661504,0.457251,0.285521,2808.849567,2441.508002,5528.454222,0.097706,5.253005e-07,-0.000182,92.285156,...,1.184065,0.593270,1.220743,0.620820,0.712082,0.518463,0.468916,0.123271,-0.013371,0.258653


In [ ]:
cols = [c for c in train.columns]

In [ ]:
 from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
train[cols] = stdsc.fit_transform(train[cols])
test[cols] = stdsc.transform(test[cols])

In [ ]:
display(train.head(3))
display(test.head(3))

,length,chroma_stft_mean,rms_mean,spectral_centroid_mean,spectral_bandwidth_mean,rolloff_mean,zero_crossing_rate_mean,harmony_mean,perceptr_mean,tempo,...,mfcc11_var,mfcc12_var,mfcc13_var,mfcc14_var,mfcc15_var,mfcc16_var,mfcc17_var,mfcc18_var,mfcc19_var,mfcc20_var
0,-0.298576,0.642278,0.177059,1.233038,1.386263,1.266091,0.764469,0.211645,0.314663,-0.234117,...,1.838597,0.833696,1.034494,0.776698,0.990850,1.481935,0.680471,0.666631,0.743034,-0.091657
1,-0.298576,0.227633,-0.595372,-0.050038,-0.362538,-0.174821,0.380594,-0.925636,-1.493513,0.877118,...,0.824710,-0.080086,0.785104,0.768889,1.027054,0.374397,0.127377,0.540703,-0.063098,0.060347
2,-0.142344,-0.050788,-0.477223,0.161218,0.571295,-0.000670,0.009428,0.213437,0.450461,-0.695707,...,0.631436,0.303991,-0.387986,0.346198,0.130285,-0.055250,-0.151197,-0.366009,-1.045659,-1.167858


,length,chroma_stft_mean,rms_mean,spectral_centroid_mean,spectral_bandwidth_mean,rolloff_mean,zero_crossing_rate_mean,harmony_mean,perceptr_mean,tempo,...,mfcc11_var,mfcc12_var,mfcc13_var,mfcc14_var,mfcc15_var,mfcc16_var,mfcc17_var,mfcc18_var,mfcc19_var,mfcc20_var
0,-0.298576,-0.648987,-1.134700,-1.172775,-0.967173,-1.282262,-0.778708,-1.049366,-0.701822,0.365950,...,1.438900,1.531189,1.933873,1.593768,1.864126,0.803889,1.817925,1.818024,1.261632,1.923743
1,-0.142344,0.703835,-1.919805,-1.723546,-1.199076,-1.699029,-1.720368,0.207379,0.356319,0.877118,...,-1.153414,-1.881063,-3.674341,-3.609437,-3.750683,-3.791344,-3.700374,-3.402824,-2.418003,-1.934408
2,-0.298576,0.939176,2.349185,0.841052,0.373789,0.601497,-0.148723,0.212788,0.200619,-0.948482,...,1.184065,0.593270,1.220743,0.620820,0.712082,0.518463,0.468916,0.123271,-0.013371,0.258653


In [ ]:
train = pd.concat([train,train_labels],axis=1)

In [ ]:
train.head()

,length,chroma_stft_mean,rms_mean,spectral_centroid_mean,spectral_bandwidth_mean,rolloff_mean,zero_crossing_rate_mean,harmony_mean,perceptr_mean,tempo,...,mfcc12_var,mfcc13_var,mfcc14_var,mfcc15_var,mfcc16_var,mfcc17_var,mfcc18_var,mfcc19_var,mfcc20_var,label
0,-0.298576,0.642278,0.177059,1.233038,1.386263,1.266091,0.764469,0.211645,0.314663,-0.234117,...,0.833696,1.034494,0.776698,0.990850,1.481935,0.680471,0.666631,0.743034,-0.091657,reggae
1,-0.298576,0.227633,-0.595372,-0.050038,-0.362538,-0.174821,0.380594,-0.925636,-1.493513,0.877118,...,-0.080086,0.785104,0.768889,1.027054,0.374397,0.127377,0.540703,-0.063098,0.060347,reggae
2,-0.142344,-0.050788,-0.477223,0.161218,0.571295,-0.000670,0.009428,0.213437,0.450461,-0.695707,...,0.303991,-0.387986,0.346198,0.130285,-0.055250,-0.151197,-0.366009,-1.045659,-1.167858,country
3,-0.142344,0.539357,0.680464,-0.445936,-0.631178,-0.478404,-0.333189,0.210407,0.310167,-0.948482,...,0.339539,-0.141871,0.382391,0.523603,-0.232210,-1.010304,-0.404582,-0.406441,-0.321155,blues
4,-0.142344,0.925586,-0.513290,0.087587,-0.601319,-0.250903,0.792784,-0.681551,-1.659057,-0.052279,...,-0.505731,-0.512941,0.509536,-0.097451,-0.645213,-0.679199,-1.395066,-0.948719,-1.377689,metal


In [ ]:
X = train.drop('label',axis=1)
y=train['label']

In [ ]:
import sklearn

In [ ]:
from sklearn.feature_selection import mutual_info_classif


In [ ]:
#mi_scores = mutual_info_classif(X,y)

In [ ]:
from sklearn.feature_selection import mutual_info_classif

def make_mi_scores(X, y):
    mi_scores = mutual_info_classif(X, y)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X, y)


In [ ]:
from sklearn.feature_selection import SelectKBest

In [ ]:
selector = SelectKBest(mutual_info_classif, k=30)

In [ ]:
selector.fit_transform(X, y)
# Get columns to keep and create new dataframe with those only
cols = selector.get_support(indices=True)
X_new = X.iloc[:,cols]

In [ ]:
X_new

,length,chroma_stft_mean,rms_mean,spectral_centroid_mean,spectral_bandwidth_mean,rolloff_mean,zero_crossing_rate_mean,perceptr_mean,mfcc1_mean,mfcc2_mean,...,rms_var,spectral_centroid_var,rolloff_var,zero_crossing_rate_var,harmony_var,perceptr_var,mfcc1_var,mfcc4_var,mfcc6_var,mfcc20_var
0,-0.298576,0.642278,0.177059,1.233038,1.386263,1.266091,0.764469,0.314663,0.493279,-0.995018,...,0.892129,1.702806,1.644748,1.464454,-0.012941,0.882548,1.052997,1.964702,1.228940,-0.091657
1,-0.298576,0.227633,-0.595372,-0.050038,-0.362538,-0.174821,0.380594,-1.493513,0.052394,0.067482,...,-0.077791,-0.399484,-0.375331,0.083251,-0.606148,-0.166932,-0.204898,0.543840,0.331641,0.060347
2,-0.142344,-0.050788,-0.477223,0.161218,0.571295,-0.000670,0.009428,0.450461,0.230404,-0.127913,...,-0.423070,0.530146,0.803409,0.242894,-0.537517,-0.078573,-0.762395,0.811495,1.013223,-1.167858
3,-0.142344,0.539357,0.680464,-0.445936,-0.631178,-0.478404,-0.333189,0.310167,0.508399,0.508329,...,0.604053,-0.371194,-0.184576,-0.295304,0.474484,0.970578,0.030053,1.027655,0.600615,-0.321155
4,-0.142344,0.925586,-0.513290,0.087587,-0.601319,-0.250903,0.792784,-1.659057,0.431285,0.169624,...,-1.304404,-1.037717,-0.806537,-0.647711,-0.691931,-0.201060,-1.028406,-0.974214,-0.193546,-1.377689
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,-0.298576,1.092797,0.197356,0.923953,0.820137,0.771998,0.633871,0.414846,0.161277,-1.063098,...,1.479820,1.747859,1.235334,1.784310,0.418598,0.913592,0.683755,0.586910,0.775770,0.254027
896,-0.142344,-1.083787,-0.641160,-1.531488,-1.487357,-1.485121,-1.241466,0.366960,-1.063685,1.008074,...,0.471335,-1.016069,-0.648327,-0.434870,-0.286069,-0.449708,0.365005,0.812267,0.851494,0.257679
897,-0.298576,0.908685,1.405492,1.279825,1.379687,1.292472,0.918037,0.138110,1.009848,-1.163855,...,2.002632,1.252163,1.241243,1.633216,0.790800,1.984006,0.578662,1.115801,1.491169,0.221268
898,-0.142344,0.704175,1.852109,-0.537285,0.527036,-0.367192,-1.245502,0.546563,0.253908,0.089727,...,2.056552,0.933623,1.690664,-0.090202,1.875894,1.464075,0.526519,0.692625,1.265211,-0.013031


In [ ]:
train = pd.concat([X_new,train_labels],axis=1)

In [ ]:
train.head()

,length,chroma_stft_mean,rms_mean,spectral_centroid_mean,spectral_bandwidth_mean,rolloff_mean,zero_crossing_rate_mean,perceptr_mean,mfcc1_mean,mfcc2_mean,...,spectral_centroid_var,rolloff_var,zero_crossing_rate_var,harmony_var,perceptr_var,mfcc1_var,mfcc4_var,mfcc6_var,mfcc20_var,label
0,-0.298576,0.642278,0.177059,1.233038,1.386263,1.266091,0.764469,0.314663,0.493279,-0.995018,...,1.702806,1.644748,1.464454,-0.012941,0.882548,1.052997,1.964702,1.228940,-0.091657,reggae
1,-0.298576,0.227633,-0.595372,-0.050038,-0.362538,-0.174821,0.380594,-1.493513,0.052394,0.067482,...,-0.399484,-0.375331,0.083251,-0.606148,-0.166932,-0.204898,0.543840,0.331641,0.060347,reggae
2,-0.142344,-0.050788,-0.477223,0.161218,0.571295,-0.000670,0.009428,0.450461,0.230404,-0.127913,...,0.530146,0.803409,0.242894,-0.537517,-0.078573,-0.762395,0.811495,1.013223,-1.167858,country
3,-0.142344,0.539357,0.680464,-0.445936,-0.631178,-0.478404,-0.333189,0.310167,0.508399,0.508329,...,-0.371194,-0.184576,-0.295304,0.474484,0.970578,0.030053,1.027655,0.600615,-0.321155,blues
4,-0.142344,0.925586,-0.513290,0.087587,-0.601319,-0.250903,0.792784,-1.659057,0.431285,0.169624,...,-1.037717,-0.806537,-0.647711,-0.691931,-0.201060,-1.028406,-0.974214,-0.193546,-1.377689,metal


In [ ]:

N_SPLITS = 5
N_REPEATS = 3
EARLY_STOPPING_ROUNDS = 300
VERBOSE = False
SEED = 17171

In [ ]:
target_column = 'label'

In [ ]:
features = [c for c in train.columns if c not in [target_column]]

In [ ]:
len(features)

30

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X = train.drop('label',axis=1)
y=train['label']
X.shape, y.shape

((900, 30), (900,))

In [ ]:
## going by the optuna way:
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
# Which hyperparameters to tune: https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/

def objective(trial):
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 101,2)
    min_samples_split = trial.suggest_float("min_samples_split", 0, 1)
    n_estimators = trial.suggest_int("n_estimators", 1, 999,2)
    max_depth = trial.suggest_int("max_depth", 1, 101,2)
    max_features = trial.suggest_categorical("max_features",["auto","sqrt"])  
    rfc = RandomForestClassifier(min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split, n_estimators=n_estimators, max_depth = max_depth, max_features = max_features)
        
    rskfold = RepeatedStratifiedKFold(n_splits=5, n_repeats=3)
    score = cross_val_score(rfc, X,y, scoring='accuracy', cv=rskfold, n_jobs = -1,error_score='raise')
    score = score.mean()
    return score

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50, timeout = 4*60*60)

[I 2022-04-22 06:57:52,603] A new study created in memory with name: no-name-385ce717-6f6c-4206-9649-5ab601559ae6
[I 2022-04-22 06:58:07,839] Trial 0 finished with value: 0.4588888888888889 and parameters: {'min_samples_leaf': 83, 'min_samples_split': 0.17848367421675249, 'n_estimators': 313, 'max_depth': 95, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.4588888888888889.
[I 2022-04-22 06:58:10,279] Trial 1 finished with value: 0.5174074074074073 and parameters: {'min_samples_leaf': 55, 'min_samples_split': 0.1549557412605317, 'n_estimators': 43, 'max_depth': 71, 'max_features': 'sqrt'}. Best is trial 1 with value: 0.5174074074074073.
[I 2022-04-22 06:58:11,009] Trial 2 finished with value: 0.26814814814814814 and parameters: {'min_samples_leaf': 9, 'min_samples_split': 0.637572392146926, 'n_estimators': 15, 'max_depth': 29, 'max_features': 'sqrt'}. Best is trial 1 with value: 0.5174074074074073.
[I 2022-04-22 06:58:46,128] Trial 3 finished with value: 0.5118518518518519 and p

In [ ]:
trial = study.best_trial      
print('Accuracy: {}'.format(trial.value))   

Accuracy: 0.7803703703703705


In [ ]:
print("Best hyperparameters: {}".format(trial.params))

Best hyperparameters: {'min_samples_leaf': 1, 'min_samples_split': 0.0051014900118787464, 'n_estimators': 823, 'max_depth': 35, 'max_features': 'sqrt'}


In [ ]:
rfc_params = trial.params

In [ ]:
rfc_params = {'min_samples_leaf': 1, 'min_samples_split': 0.0051014900118787464, 'n_estimators': 823, 'max_depth': 35, 'max_features': 'sqrt'}

In [ ]:
test = test[features]

In [ ]:
test.shape

(100, 30)

In [ ]:

from sklearn.metrics import recall_score,precision_score,f1_score
from sklearn.metrics import classification_report
import statistics
from statistics import mean

In [ ]:
%%time
N_SPLITS = 5
y_preds = []
acc= []
from sklearn.model_selection import StratifiedKFold

folds = StratifiedKFold(n_splits = N_SPLITS)

for fold, (train_id, valid_id) in enumerate(folds.split(X,y)):
  X_train = X.iloc[train_id]
  y_train = y.iloc[train_id]
  X_valid = X.iloc[valid_id]
  y_valid = y.iloc[valid_id]
  model = RandomForestClassifier(**rfc_params)
  
  model.fit(X_train, y_train)
  valid_pred = model.predict(X_valid)

#print('fold:{}'.format(fold+1))
  #print('Validation Accuracy score : {}'.format(accuracy_score(y_valid, valid_pred)))
  #print('-'*100)
  acc.append(accuracy_score(y_valid, valid_pred))

  y_preds.append(model.predict(test))



CPU times: user 17 s, sys: 80.3 ms, total: 17.1 s
Wall time: 17.6 s


In [ ]:
print('the mean accuracy is: {}'.format(mean(acc)))


the mean accuracy is: 0.78


In [ ]:
from collections import Counter
def mode(data):
    """Return the most common data point from discrete or nominal data.
    ``mode`` assumes discrete data, and returns a single value. This is the
    standard treatment of the mode as commonly taught in schools:
        >>> mode([1, 1, 2, 3, 3, 3, 3, 4])
        3
    This also works with nominal (non-numeric) data:
        >>> mode(["red", "blue", "blue", "red", "green", "red", "red"])
        'red'
    If there are multiple modes with same frequency, return the first one
    encountered:
        >>> mode(['red', 'red', 'green', 'blue', 'blue'])
        'red'
    If *data* is empty, ``mode``, raises StatisticsError.
    """
    pairs = Counter(iter(data)).most_common(1)
    try:
        return pairs[0][0]
    except IndexError:
        raise StatisticsError('no mode for empty data') from None




In [ ]:
arr = [y_preds[0][0],y_preds[1][0], y_preds[2][0], y_preds[3][0], y_preds[4][0]]

In [ ]:
y_pred = pd.DataFrame(columns=['label'])
pred= []
for j in range(0,100):
    arr = [y_preds[0][j], y_preds[1][j], y_preds[2][j], y_preds[3][j],y_preds[4][j]]
    pred.append(mode(arr))

In [ ]:
y_pred["label"]=pred

In [ ]:
y_pred.to_csv('rfc_mutual_pred.csv', index=False)

In [ ]:
!cp -r '/content/rfc_mutual_pred.csv' /content/drive/MyDrive/project9/FS_FeatSelect



In [ ]:
X_new.to_csv('train_mutual.csv',index=False)
test.to_csv('test_mutual.csv',index=False)

In [ ]:
!cp -r '/content/train_mutual.csv' /content/drive/MyDrive/project9/FS_FeatSelect




In [ ]:
!cp -r '/content/test_mutual.csv' /content/drive/MyDrive/project9/FS_FeatSelect




In [ ]:
y_pred

,label
0,reggae
1,classical
2,hiphop
3,country
4,rock
...,...
95,reggae
96,disco
97,pop
98,pop
